### Try pretrained model:

#### Load model, tokenizer and labels

In [4]:
import os
import pickle
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

df = pd.read_csv('data/emotion.data.zip', compression='zip')

base = os.path.abspath(os.path.dirname(__name__))

# Load pretrained model
model = tf.keras.models.load_model(f"{base}/data/model_att_v1.h5")

# Load tokenizer.
with open('data/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
# Load id2label dictionary.
with open('data/id2label.pickle', 'rb') as handle:
    id2label = pickle.load(handle)



#### Choose random text and label and test.

In [5]:
max_length = 178
trunc_type = "post"
i = random.randint(0, len(df["text"]) -1)

sample_text = df["text"][i]
sample_label = df["emotions"][i]

sequences = tokenizer.texts_to_sequences([sample_text])

padded = pad_sequences(
    sequences, 
    maxlen=max_length, 
    padding=trunc_type, 
    truncating=trunc_type
)

y_pred = model.predict(padded)
print(sample_text,"\n") 
print(f"{sample_label} VS {id2label[np.argmax(y_pred)]}")

im feeling really contented 

joy VS joy


In [6]:
text = "there was a little cute dog."
sequences = tokenizer.texts_to_sequences([text])

padded = pad_sequences(
    sequences, 
    maxlen=max_length, 
    padding=trunc_type, 
    truncating=trunc_type
)

id2label[np.argmax(model.predict(padded))]

'joy'

**Note:** Here you may put your own text.

### Train your own model.

In [ ]:
import random
import numpy as np
import tensorflow as tf


physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass


seed = 1403
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

### Load Data:

In [ ]:
import pandas as pd

df = pd.read_csv('data/emotion.data.zip', compression='zip')
df.head(10)

### Plot counts of each class:

In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(6, 4))
df.emotions.value_counts().plot.bar()
plt.title("count/class")
plt.show()

### WordCloud:

In [ ]:
from data.utils import word_cloud

X, y = df["text"].values.tolist(), df["emotions"].values.tolist()
word_cloud(" ".join(X))

### Tokenize, padding:

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from data.utils import hist_plot

vocab_size = 10000
embedding_dim = 100
trunc_type = "post"
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X)

# Saving tokenizer
with open('data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X)

max_length = max(list(map(len, sequences)))

padded = pad_sequences(sequences, maxlen=max_length, padding=trunc_type)
hist_plot(np.count_nonzero(padded, axis=1), "Padded sentences", "Length")

### Process Data:

In [ ]:
from sklearn.model_selection import train_test_split
from data.utils import pie_plot

embedding_dim = 100

label2id = {l: i for i, l in enumerate(set(y))}
id2label = {v: k for k, v in label2id.items()}

# Saving id2label
with open('data/id2label.pickle', 'wb') as handle:
    pickle.dump(id2label, handle)

y = tf.keras.utils.to_categorical(
    [label2id[i] for i in y], 
    num_classes=len(label2id), 
    dtype='float32'
)


split = 0.33

x_train, x_test, y_train, y_test = train_test_split(padded, y, test_size=split)

pie_plot(data=[len(x_train), len(x_test)], labels=['Train', 'Test'])

### Network:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Dropout, Dense, TimeDistributed,
    Bidirectional, LSTM, Dot, Activation, Reshape
    
)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping

K.clear_session()


inputs = Input(shape=(max_length,), dtype='int32', name="inputs")
emb = Embedding(vocab_size, embedding_dim, input_length=max_length, name="embeddings")(inputs)
emb = Dropout(0.2)(emb)

lstm = Bidirectional(LSTM(embedding_dim, return_sequences=True))(emb)
lstm = Dropout(0.2)(lstm)

att = TimeDistributed(Dense(1))(lstm)
att = Reshape((max_length,))(att)
att = Activation('softmax', name='att_vector')(att)
att = Dot(axes=1, name="att_output")([lstm, att])


dense = Dense(embedding_dim, activation='relu')(att)
outputs = Dense(len(label2id), activation='softmax', name="outputs")(dense)

model = Model(inputs, outputs, name="sematic_with_attention")
plot_model(model, "model.png", show_shapes=True, dpi=60)

In [ ]:

callback = EarlyStopping(monitor='val_loss', patience=2)


model.compile(
    loss="categorical_crossentropy", 
    metrics=["accuracy"], 
    optimizer='adam'
)


history = model.fit(
    x_train, 
    y_train, 
    epochs=100, 
    batch_size=64, 
    validation_data=(x_test, y_test), 
    callbacks=[callback]
)


model.save("data/model_att_v1.h5")